Enoncé du TD (Analyse de sentiment)
===================
Le TD consiste à finir l'implémentation d'un prédicteur naïf de la polarité d'une critique de film tirée de IMDB 
anglais. Le modèle est un modèle de régression logistique traditionnel mais implémenté avec pytorch.

Le jeu de données est à télécharger depuis le site suivant : http://ai.stanford.edu/~amaas/data/sentiment/

Le TD consiste à compléter les trous laissés dans ce notebook :

* Remplacer les chemins d'accès aux données codés en dur dans le notebook
* Implémenter le chargement de données de test dans le notebook dans la cellule marquée à cet effet
* Compléter la méthode `run_test(...)` de la classe `SentimentAnalyser` pour qu'elle renvoie un score d'accurracy de classification sur un jeu de test.
* Découper les données de test en données de validation (15000 premiers exemples) et de test final (10000 derniers exemples)
* Augmenter la méthode `train(...)` de la classe `SentimentAnalyser` pour qu'elle prenne en second argument les données de validation. Le corps de la méthode sera modifié pour (1) réaliser une évaluation sur les données de validation à chaque époque (2) sauvegarder au final le modèle qui minimise la perte sur les données de validation. 
* Faire une recherche d'hyper-paramètres (nombre d'épochs et learning rate).

Le rendu attendu est une copie de ce notebook qui aura été complétée. Profitez du framework notebook pour commenter vos réponses si besoin.




In [ ]:
import os
import os.path
import random
from collections import Counter


random.seed(1)


#loads data from disk
def load_dataset(dir_path,ref_label):
    """
    Loads a dataset from a directory path and 
    returns a list of couples (Counter of Bow_freq,ref_label) one for each text
    """
    dpath    = os.path.abspath(dir_path)
    data_set = [] 
    for f in os.listdir(dpath):
        filepath    = os.path.join(dpath,f)
        file_stream = open(filepath)
        text        = file_stream.read().split()
        file_stream.close()
        data_set.append((Counter(text),ref_label))
    return data_set
    
trainpos = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/train/pos",1)
trainneg = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/train/neg",0)
trainset  = trainpos
trainset.extend(trainneg)
random.shuffle(trainset)

In [ ]:
testpos = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/test/pos",1)
testneg = load_dataset("/Users/bcrabbe/parsing-at-diderot/data/aclImdb/test/neg",0)
testset = testpos
testset.extend(testneg)
random.shuffle(testset)

In [ ]:
#creates a dict that maps each known word string to a unique integer
def make_w2idx(dataset):
    wordset = set([])
    for X,Y in dataset:
        wordset.update([word for word in X])
    return dict(zip(wordset,range(len(wordset))))   

def vectorize_text(counter,w2idx):
    xvec = torch.zeros(len(w2idx))
    for word in counter:
        if word in w2idx:       #manages unk words (ignored)
            xvec[w2idx[word]] = counter[word] 
    return xvec.squeeze()

def vectorize_target(ylabel):
     return torch.tensor(float(ylabel))
  

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class SentimentAnalyzer(nn.Module): 
        def __init__(self):
            
            super(SentimentAnalyzer, self).__init__()
            self.reset_structure(1,1)
            
        def reset_structure(self,vocab_size, num_labels):
            
            self.W = nn.Linear(vocab_size, num_labels)
            
        def forward(self, text_vec):
            return F.sigmoid(self.W(text_vec)) #sigmoid is the logistic activation
        
        def train(self,train_set,dev_set,learning_rate,epochs):
            self.w2idx = make_w2idx(train_set)
            self.reset_structure(len(self.w2idx),1)
            loss_func  = nn.BCELoss() #remind that minimizing Binary Cross Entropy <=> minimizing NLL
            optimizer  = optim.SGD(self.parameters(), lr=learning_rate)
            
            min_loss = np.iinfo(np.int32).max
            for epoch in range(epochs):
                global_logloss = 0.0
                for X, Y in train_set: 
                    self.zero_grad()
                    xvec            = vectorize_text(X,self.w2idx)
                    yvec            = vectorize_target(Y)
                    prob            = self(xvec).squeeze()
                    loss            = loss_func(prob,yvec)
                    loss.backward()
                    optimizer.step()
                    global_logloss += loss.item()
                print("Epoch %d, mean cross entropy (train) = %f"%(epoch,global_logloss/len(train_set)))

                with torch.no_grad():
                    global_logloss = 0.0
                    for X, Y in dev_set: 
                        xvec            = vectorize_text(X,self.w2idx)
                        yvec            = vectorize_target(Y)
                        prob            = self(xvec).squeeze()
                        loss            = loss_func(prob,yvec)
                        global_logloss += loss.item()
                    print("Epoch %d, mean cross entropy (dev)   = %f\n"%(epoch,global_logloss/len(dev_set)))
                    if global_logloss < min_loss:
                        torch.save(self.state_dict(), 'sentiment_model.wt')
            self.load_state_dict(torch.load('sentiment_model.wt'))
            
        def run_test(self,test_set):
            
            ncorrect = 0
            with torch.no_grad():
                for X, Y in test_set: 
                    xvec            = vectorize_text(X,self.w2idx)
                    prob            = self(xvec).squeeze()
                    if Y == 1 and prob > 0.5:
                        ncorrect += 1
                    if Y == 0 and prob <= 0.5: 
                        ncorrect += 1
            print("Test Accurracy",ncorrect/len(test_set))
            

In [ ]:
sent = SentimentAnalyzer()
sent.train(trainset,validset,0.1,10)

In [ ]:
sent.run_test(testset)